In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display
import gc

In [2]:
test_data = pd.read_feather("data/test_data.ftr")

In [3]:
null_cols = test_data.loc[:, test_data.isnull().sum() > 0].columns

In [4]:
for i, col in enumerate(null_cols):
    print(f"{i+1}/{len(null_cols)}", end="\r")
    if test_data[col].dtype == np.float16 or test_data[col].dtype == int:
        test_data.loc[test_data[col].isnull(), col] = test_data[col].notnull().mean()

In [5]:
test_num_data = test_data._get_numeric_data()

In [6]:
del test_data, col, i, null_cols

In [7]:
gc.collect()

0

In [8]:
test_num_data = test_num_data.to_numpy()

In [9]:
gc.collect()

0

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
scaler.fit(test_num_data)

MinMaxScaler()

In [14]:
gc.collect()

0

In [13]:
for i in range(test_num_data.shape[0]):
    print(f"{i+1}/{test_num_data.shape[0]}", end="\r")
    test_num_data[i] = scaler.transform(test_num_data[i].reshape(1,-1)).reshape(-1)

In [19]:
np.save("data/x_test.npy", test_num_data)

In [2]:
x_test = np.load("data/x_test.npy")

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
from keras import models

In [5]:
model = models.load_model("model.hdf5")

In [6]:
x_test.shape

(11363762, 186)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 186)               0         
                                                                 
 dense (Dense)               (None, 256)               47872     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 1)                 1

In [8]:
gc.collect()

252

In [9]:
prediction = model.predict(x_test, batch_size=10000)

1137/1137 [==============================] - 18s 15ms/step


In [12]:
np.save("data/prediction.npy", prediction.reshape(-1))

In [4]:
test_data = pd.read_feather("data/test_data.ftr")

In [5]:
prediction = np.load("data/prediction.npy")

In [6]:
prediction = np.where(prediction <= 0.5, 0, 1)

In [7]:
prediction

array([0, 0, 0, ..., 0, 0, 0])

In [8]:
submission = pd.DataFrame({
    "customer_ID": test_data.customer_ID.values,
    "prediction": prediction
}).set_index("customer_ID")

In [30]:
customer_ID, prediction_mean = [], []
for i, cus_id in enumerate(submission.index.unique()):
    print(f"{i+1}/924621", end="\r")
    customer_ID.append(cus_id)
    prediction_mean.append(0 if submission.loc[cus_id].prediction.mean() <= 0.5 else 1)

In [33]:
submission_unique = pd.DataFrame({
    "customer_ID": customer_ID,
    "prediction": prediction_mean
}).set_index("customer_ID")

In [34]:
submission_unique

,prediction
customer_ID,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,1
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,1
...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,1
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,1


In [35]:
submission_unique.to_csv("submission.csv")